# Example with ARGO data

We first need to download some ARGO data for example.

In [ ]:
import requests

url = "https://www.ncei.noaa.gov/data/oceans/argo/gadr/data/coriolis/69022/nodc_69022_prof.nc"
r = requests.get(url, allow_redirects=True)
with open("ARGO_example.nc", "wb") as f:
    f.write(r.content)

In [ ]:
# ruff: noqa: F401
# ruff: noqa: I001
import xarray as xr
import cf_xarray
import gsw_xarray as gsw

In [ ]:
ds = xr.open_dataset("ARGO_example.nc")

In [ ]:
ds

We can rely on cf-xarray to see what variables have standard names in our dataset:

In [ ]:
ds.cf

The dataset contains multiple time the same variable (e.g. 'pres_adjusted' and 'pres' both have the standard name 'sea_water_pressure'). For the accessor to work, only 1 variable or each standard name must be present, explicitely stated when calling the function, or the gsw option `set_cf_name_preference` must be set.
For this example we will only retain the adjusted variables. We set the global option, but we could also use context, i.e.

In [ ]:
with gsw.set_cf_name_preference(
    sea_water_pressure="pres_adjusted",
    sea_water_practical_salinity="psal_adjusted",
    sea_water_temperature="temp_adjusted",
):
    # do the computation
    pass

gsw.set_cf_name_preference(
    sea_water_pressure="pres_adjusted",
    sea_water_practical_salinity="psal_adjusted",
    sea_water_temperature="temp_adjusted",
)

# We can check the options we have set:
gsw.get_options()

In the following sections we will demonstrate each features of gsw-xarray. We will focus on computing the potential density anomaly.

## Basic usage as drop in replacement of gsw

In [ ]:
help(gsw.sigma0)

We need Absolute Salinity and Conservative Temperature, so 1st we need to do some conversions:

In [ ]:
SA = gsw.SA_from_SP(
    SP=ds.psal_adjusted, p=ds.pres_adjusted, lon=ds.longitude, lat=ds.latitude
)
CT = gsw.CT_from_t(SA=SA, t=ds.temp_adjusted, p=ds.pres_adjusted)
sigma0 = gsw.sigma0(SA=SA, CT=CT)
sigma0

## Using Pint and pint-xarray to handle units

In [ ]:
import pint_xarray
import cf_xarray.units

In [ ]:
ds_pint = ds.pint.quantify()
ds_pint

We compute again sigma0, using the `ds_pint` dataset, i.e. variables have a physical dimension

In [ ]:
SA = gsw.SA_from_SP(
    SP=ds_pint.psal_adjusted,
    p=ds_pint.pres_adjusted,
    lon=ds_pint.longitude,
    lat=ds_pint.latitude,
)
CT = gsw.CT_from_t(SA=SA, t=ds_pint.temp_adjusted, p=ds_pint.pres_adjusted)
sigma0 = gsw.sigma0(SA=SA, CT=CT)
sigma0

gsw-xarray converts the units (if necessary) when using pint quantities:

In [ ]:
# start to convert the pressure into Pascal
pressure_in_pascal = ds_pint.pres_adjusted.pint.to("Pa")
pressure_in_pascal

Compute again density, using the pressure in Pascal. No worries as the conversion to dbar is automatic!

In [ ]:
SA = gsw.SA_from_SP(
    SP=ds_pint.psal_adjusted,
    p=pressure_in_pascal,
    lon=ds_pint.longitude,
    lat=ds_pint.latitude,
)
CT = gsw.CT_from_t(SA=SA, t=ds_pint.temp_adjusted, p=ds_pint.pres_adjusted)
sigma0 = gsw.sigma0(SA=SA, CT=CT)
sigma0

## Using the accessor to simplify the workflow
### Common case

gsw-xarray adds ths `gsw` accessor to datasets. This accessor makes it easy to run the gsw functions on variables from a dataset.

A first solution is to use the accessor exactly as when using gsw:

In [ ]:
ds.gsw.SA_from_SP(
    SP=ds.psal_adjusted, p=ds.pres_adjusted, lon=ds.longitude, lat=ds.latitude
)

This is however not very useful... A better option is to simply give the name of the variables from the dataset:

In [ ]:
ds.gsw.SA_from_SP(
    SP="psal_adjusted", p="pres_adjusted", lon="longitude", lat="latitude"
)

It is even possible to go 1 step further and rely on the usage of standard name! In this case, you don't need to provide any argument for the variables with the proper standard name.

With this method, it is way faster to compute the density:

In [ ]:
# WITHOUT any detection
SA = gsw.SA_from_SP(
    SP=ds.psal_adjusted, p=ds.pres_adjusted, lon=ds.longitude, lat=ds.latitude
)
CT = gsw.CT_from_t(SA=SA, t=ds.temp_adjusted, p=ds.pres_adjusted)
sigma0 = gsw.sigma0(SA=SA, CT=CT)

# WITH autodetection
ds = ds.merge(ds.gsw.SA_from_SP())
ds = ds.merge(ds.gsw.CT_from_t())
ds = ds.merge(ds.gsw.sigma0())

You can also use brackets to get either 1 or multiple variables computed:

In [ ]:
ds.gsw["sigma0"]  # Returns a DataArray
ds.gsw[["sigma0"]]  # Returns a Dataset
ds.gsw[["sigma0", "alpha", "beta", "sigma1", "rho"]]  # With multiple outputs

Of course any kind of mixture between all the solutions is possible:

In [ ]:
# Give a value for SP
# Take p from dataset
# Automatically get lon and lat based on standard names
ds.gsw.SA_from_SP(SP=35, p="pres_adjusted")

And it is also possible to use automatic discovery of argument with pint datasets:

In [ ]:
ds_pint.gsw.SA_from_SP()

### Case with some argument without standard name

Some functions have argument without any standard name. In this case, it is possible to refer to these arguments using gsw-xarray options.

Let's take the function `gsw.SP_salinometer` that has 2 arguments: `Rt` without standard name, and `t` the in situ temperature.

A 1st option is to explicitely provide a value or the name from the dataset (we will create some fake data for the purpose of this example):

In [ ]:
ds["salinometer_Rt"] = 35

In [ ]:
ds.gsw.SP_salinometer(Rt="salinometer_Rt")

A 2nd solution is to use gsw-xarray option with function `set_non_cf_name`. This way if you need to compute multiple times functions that use these arguments without a standard name, you only need to provide once the mapping.

In [ ]:
help(gsw.set_non_cf_name)

Using a context manager:

In [ ]:
with gsw.set_non_cf_name(Rt="salinometer_Rt"):
    ds.gsw.SP_salinometer()

Or globally:

In [ ]:
gsw.set_non_cf_name(Rt="salinometer_Rt")
ds.gsw.SP_salinometer()

## Note on performance

It is still very slightly faster to provide the arguments than rely on autodetect, however for large data sets this difference should be negligible compared to the internal computation time of the gsw functions.

In [ ]:
%%timeit
gsw.SA_from_SP(
    SP=ds.psal_adjusted, p=ds.pres_adjusted, lon=ds.longitude, lat=ds.latitude
)

In [ ]:
%%timeit
ds.gsw.SA_from_SP(
    SP="psal_adjusted", p="pres_adjusted", lon="longitude", lat="latitude"
)

In [ ]:
%%timeit
ds.gsw.SA_from_SP()

Compare with upstream gsw

In [ ]:
import gsw as gsw_upstream

In [ ]:
%%timeit
gsw_upstream.SA_from_SP(
    SP=ds.psal_adjusted, p=ds.pres_adjusted, lon=ds.longitude, lat=ds.latitude
)